# Vamos utilizar o mlflow para versionar a nossa model pipeline

Este notebook vai ser muito similar ao notebook `11_random_forests_mlflow.ipynb`, com a diferença que aqui vamos registar a nossa pipeline inteira (transformação de dados + modelo) em vez de apenas o modelo.

Referências:
* [Model Registry](https://www.mlflow.org/docs/latest/model-registry.html#concepts)
* [sklearn Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html)

Para perceber o que muda neste notebook será preciso percorrer o mesmo, e as alterações face ao notebook anterior irão ter um comentário em markdown antes das mesmas

In [1]:
import mlflow

In [2]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

In [3]:
ROOT_PATH = '../../data/'
SEED = 42
TARGET_COL = "Outcome"

In [4]:
mlflow.set_tracking_uri("sqlite:///../../mlruns/mlflow.db")

In [5]:
from pathlib import Path

artifact_location = Path("../../mlruns/db")

# criar a pasta ../../mlruns/db caso ela não exista
artifact_location.mkdir(parents=True, exist_ok=True)

try:
    mlflow.create_experiment(name="Diabetes Prediction Experiment", artifact_location=artifact_location.as_posix())
except mlflow.MlflowException:
    # experiência já foi criada, só precisamos de fazer set dela
    pass

mlflow.set_experiment(experiment_name="Diabetes Prediction Experiment")

<Experiment: artifact_location=('file:///c:/Users/gilso/OneDrive/Área de '
 'Trabalho/rumos/notebooks/random_forest/../../mlruns/db'), creation_time=1701106626230, experiment_id='1', last_update_time=1701106626230, lifecycle_stage='active', name='Diabetes Prediction Experiment', tags={}>

In [6]:
train_path = ROOT_PATH + 'diabetes_train.csv'
test_path = ROOT_PATH + 'diabetes_test.csv'

train_set = pd.read_csv(train_path)
test_set = pd.read_csv(test_path)

In [10]:
train_dataset = mlflow.data.from_pandas(train_set, source=train_path, targets=TARGET_COL, name="Diabetes Train Dataset")
test_dataset = mlflow.data.from_pandas(test_set, source=test_path, targets=TARGET_COL, name="Diabetes Test Dataset")

c:\Users\gilso\anaconda3\lib\site-packages\mlflow\data\dataset_source_registry.py:143: UserWarning: The specified dataset source can be interpreted in multiple ways: LocalArtifactDatasetSource, LocalArtifactDatasetSource. MLflow will assume that this is a LocalArtifactDatasetSource source.
  return _dataset_source_registry.resolve(
c:\Users\gilso\anaconda3\lib\site-packages\mlflow\data\digest_utils.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  string_columns = trimmed_df.columns[(df.applymap(type) == str).all(0)]
c:\Users\gilso\anaconda3\lib\site-packages\mlflow\data\dataset_source_registry.py:143: UserWarning: The specified dataset source can be interpreted in multiple ways: LocalArtifactDatasetSource, LocalArtifactDatasetSource. MLflow will assume that this is a LocalArtifactDatasetSource source.
  return _dataset_source_registry.resolve(
c:\Users\gilso\anaconda3\lib\site-packages\mlflow\data\digest_utils.py:26: FutureWarning: DataFrame.ap

In [7]:
mlflow.start_run(run_name="Random Forest Pipeline Run")

<ActiveRun: >

In [8]:
# Guardamos a SEED utilizado como parametro

mlflow.log_param("seed", SEED)

42

In [11]:
# Neste ponto guardarmos o dataset de treino e de teste associado à run

mlflow.log_input(train_dataset, context="train")
mlflow.log_input(test_dataset, context="test")

c:\Users\gilso\anaconda3\lib\site-packages\mlflow\data\pandas_dataset.py:134: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  return _infer_schema(self._df)


In [12]:
X_train = train_set.drop([TARGET_COL], axis = 1)
y_train = train_set[TARGET_COL]

X_test = test_set.drop([TARGET_COL], axis = 1)
y_test = test_set[TARGET_COL]

In [13]:
X_train.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,3,173,78,39,185,33.8,0.970,31
1,6,134,80,37,370,46.2,0.238,46
2,5,104,74,0,0,28.8,0.153,48
3,1,139,46,19,83,28.7,0.654,22
4,5,137,108,0,0,48.8,0.227,37


# Model Pipeline

É nesta secção que estará a maior difenreça neste notebook.

Ou invés de logarmos separadamente o scaler e o modelo, **vamos juntar tudo através de uma `Pipeline` e logar a pipeline como um todo!**

Assim não vamos cair no erro de utilizar o modelo sem a parte de processamento de dados ser previamente aplicamente atrás, o que já vimos no notebook anterior que induz em previsões erradas

In [14]:
rf_pipeline = Pipeline([('scaler', StandardScaler()), ('random_forest', RandomForestClassifier(random_state=SEED))])

In [15]:
rf_pipeline.fit(X_train, y_train)

c:\Users\gilso\anaconda3\lib\site-packages\sklearn\utils\validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Users\gilso\anaconda3\lib\site-packages\sklearn\utils\validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():


Pipeline(steps=[('scaler', StandardScaler()),
                ('random_forest', RandomForestClassifier(random_state=42))])

In [16]:
# vamos guardar a nossa fitted random forest pipeline (que inclui o scaler + o RandomForestClassifier) 
mlflow.sklearn.log_model(rf_pipeline, artifact_path="random_forest_pipeline")

c:\Users\gilso\anaconda3\lib\site-packages\_distutils_hack\__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [17]:
y_preds = rf_pipeline.predict(X_test)

c:\Users\gilso\anaconda3\lib\site-packages\sklearn\utils\validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():


In [18]:
accuracy_score(y_test, y_preds)

0.7368421052631579

In [19]:
feature_imp = pd.Series(rf_pipeline.steps[1][1].feature_importances_, index = X_train.columns).sort_values(ascending = False)
feature_imp

Glucose                     0.258462
BMI                         0.180059
Age                         0.144415
DiabetesPedigreeFunction    0.106236
BloodPressure               0.090174
Pregnancies                 0.078640
SkinThickness               0.072315
Insulin                     0.069700
dtype: float64

In [20]:
feature_imp_path = "../../data/feature_importance.csv"
feature_imp.to_csv(feature_imp_path)
mlflow.log_artifact(feature_imp_path)

In [21]:
rf_pipeline = Pipeline([('scaler', StandardScaler()), ('random_forest', RandomForestClassifier(random_state=SEED))])

# Os parametros de um dado step da Pipeline podem ser especificados no parameters do GridSearchCV utilizando o '__' a separar 
# i.e. "{nome_do_step}__{nome_do_parametro}" 
# https://scikit-learn.org/stable/tutorial/statistical_inference/putting_together.html
parameters = {'random_forest__n_estimators': [10, 100, 300]}

clf = GridSearchCV(rf_pipeline, parameters, cv = 5).fit(X_train, y_train)

c:\Users\gilso\anaconda3\lib\site-packages\sklearn\utils\validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Users\gilso\anaconda3\lib\site-packages\sklearn\utils\validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Users\gilso\anaconda3\lib\site-packages\sklearn\utils\validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Users\gilso\anaconda3\lib\site-packages\sklearn\utils\validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version

In [22]:
mlflow.sklearn.log_model(clf, artifact_path="grid_search_cv")

In [23]:
tunned_rf_pipeline = clf.best_estimator_

## Tunned Random Forest Model Pipeline



Agora que temos a melhor configuração de parametros para a nossa pipeline de random forest vamos regista-la!

A maneira como o fazemos é exatamente como faziamos no notebook 11, e **vamos registar esta pipeline com o mesmo 
nome com que registamos o modelo no notebook 11, o que vais criar apenas uma nova versão deste model no model registry**

In [24]:
mlflow.sklearn.log_model(tunned_rf_pipeline, "tunned_rf_pipeline", registered_model_name="random_forest")

Registered model 'random_forest' already exists. Creating a new version of this model...
Created version '2' of model 'random_forest'.


In [25]:
tunned_rf_pipeline.get_params()

{'memory': None,
 'steps': [('scaler', StandardScaler()),
  ('random_forest', RandomForestClassifier(random_state=42))],
 'verbose': False,
 'scaler': StandardScaler(),
 'random_forest': RandomForestClassifier(random_state=42),
 'scaler__copy': True,
 'scaler__with_mean': True,
 'scaler__with_std': True,
 'random_forest__bootstrap': True,
 'random_forest__ccp_alpha': 0.0,
 'random_forest__class_weight': None,
 'random_forest__criterion': 'gini',
 'random_forest__max_depth': None,
 'random_forest__max_features': 'auto',
 'random_forest__max_leaf_nodes': None,
 'random_forest__max_samples': None,
 'random_forest__min_impurity_decrease': 0.0,
 'random_forest__min_samples_leaf': 1,
 'random_forest__min_samples_split': 2,
 'random_forest__min_weight_fraction_leaf': 0.0,
 'random_forest__n_estimators': 100,
 'random_forest__n_jobs': None,
 'random_forest__oob_score': False,
 'random_forest__random_state': 42,
 'random_forest__verbose': 0,
 'random_forest__warm_start': False}

In [26]:
y_preds = tunned_rf_pipeline.predict(X_test)

c:\Users\gilso\anaconda3\lib\site-packages\sklearn\utils\validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():


In [27]:
acc = accuracy_score(y_test, y_preds)

In [28]:
mlflow.log_metric("accuracy", acc)

In [29]:
mlflow.end_run()

## Opcional - Ver o modelo registado na UI do mlflow

A UI do mlflow permite ver de forma visual todos os modelos registados até ao momento, qual é a útlima versão do mesmo e a que runs estão associados.

Para correr a UI do mflow, tendo nós o modelo registado na db, temos que especificar esta db como a "backend store" em que temos os metados.

Para correr a UI do mflow é necessário executar, na Anaconda Prompt na raiz deste projeto (pasta rumos) e tendo activo o ambiente utilizado neste projeto, o comando:

`mlflow ui --backend-store-uri sqlite:///./mlruns/mlflow.db`

**Nota:** O comando em cima irá iniciar a UI de mlflow na porta 5000. Caso queiram mudar esta porta devem acrescentar `--port <PORT>` ao comando (em que <PORT> deve ser substituido pela porta desejada). 

O comando acima não irá funcionar caso tenham tido alguns problemas no Windows com a instalação do mlflow, mas como o título desta secção indica este passo é apenas opcional e não irá ser avaliado.

Após executarem este comando, vão poder ver a UI do mlflow no vosso browser acedendo a 

`localhost:5000`

(se tiverem alterado a porta em que o mlflow UI é iniciado então devem de alterar também aqui o 5000 por essa porta)

Na tab de `Models` podem explorar os modelos que registaram.